### Input files: Price-Today, orders-log
### Output files: Price-for-Order, orders-log, monthly-sales

In [2]:
import glob
import os
import pandas as pd
from datetime import date, timedelta, datetime
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','amount':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}

pd.options.display.max_rows = 50
pd.options.display.float_format = '{:.2f}'.format

today = date.today()
yesterday = today - timedelta(days=1)
print(today, yesterday)

2025-02-28 2025-02-27


In [3]:
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2025-02-28
yesterday: 2025-02-27


In [4]:
# If run after actual work day
#today = today - timedelta(days=2)
print(today)

2025-02-28


In [5]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-02-28 20:47:07


### Restart and Run All Cells

In [7]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [8]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


## Must rerun whenever there is change in orders

In [10]:
sql = """
SELECT trade, name, qty, price AS target, active, reason, market, xdate 
FROM orders
ORDER BY trade, name"""
orders = pd.read_sql(sql, conlite)
orders.shape

(23, 8)

In [11]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [12]:
prices = pd.read_csv(input_file)
prices.shape

(182, 10)

In [13]:
df_merge  = pd.merge(orders, prices, on = 'name', how='inner')
df_merge.shape

(23, 17)

In [14]:
df_merge['spd'] = 0

In [15]:
colo = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'price', 'change', '%change',
        'active', 'xdate']
df_merge[colo].shape

(23, 12)

In [16]:
file_name = "Price-for-Order.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [17]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\Price-for-Order.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\Price-for-Order.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\Price-for-Order.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\Price-for-Order.csv


In [18]:
df_merge[colo].to_csv(output_file, header=True, index=False)
df_merge[colo].to_csv(god_file, header=True, index=False)
df_merge[colo].to_csv(icd_file, header=True, index=False)
df_merge[colo].to_csv(osd_file, header=True, index=False)

### call ruby ruby\order-log-new.rb

In [20]:
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'

In [21]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [22]:
!ruby ruby\\order-log-new.rb

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
B,3BBIF,-2,RD50Pct,SET,10000,5.25,5.35,0.05,0.943396,1,2025-02-18
B,AIMIRT,-4,RND05pct,SET,5000,10.2,10.6,0.0,0.0,1,2024-11-26
B,CPNREIT,-9,RD40Pct,SET,5000,10.8,11.7,-0.2,-1.680672,1,2025-02-27
B,DIF,-23,RD50Pct,SET,5000,6.7,7.85,-0.15,-1.875,1,2022-02-13
B,GVREIT,-7,RND10Pct,SET,5000,6.2,6.55,-0.05,-0.757576,1,2022-02-26
B,JMART,-10,RND75pct,SET50,3200,8.95,9.45,-0.2,-2.072539,1,2022-02-25
B,KCE,-43,RD80Pct,SET100,1000,14.0,18.3,-1.4,-7.106599,1,2022-08-26
B,MCS,-26,RD60Pct,SET,5000,5.8,7.1,-0.05,-0.699301,0,2025-04-18
B,NER,-13,RD40pct,SET,6000,4.7,4.96,0.02,0.404858,0,2025-04-24
B,RCL,-31,RD45pct,SET100,3000,21.25,24.4,-0.6,-2.4,1,2022-03-12
B,SENA,-18,RD60pct,SET,7000,1.92,2.2,0.0,0.0,0,2024-05-10
B,STA,-8,RD50Pct,SET100,2500,15.0,15.8,0.0,0.0,1,2025-04-18
B,TFFIF,-6,RD20Pct,SET,5000,6.0,6.3,0.05,0.8,1,2025-02-27
B,WHAIR,-5,RD30pct,SET,5000,5.4,5.65,-0.05,-0.877193,1,2022-03-03
S,AWC,102,COS,SET50,9000,4.9

In [23]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'

In [24]:
file_name   = 'orders-log.csv'
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\orders-log.csv


In [25]:
df = pd.read_csv(input_file, sep=',', index_col=None)
df['amount'] = df['qty'] * df['target']
df.query('spd >= -3 and spd <= 3')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
0,B,3BBIF,-2,RD50Pct,SET,10000,5.25,5.35,0.05,0.94,1,2025-02-18,52500.00


In [26]:
df.query('active == 2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [27]:
df.query('active == 1')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
0,B,3BBIF,-2,RD50Pct,SET,10000,5.25,5.35,0.05,0.94,1,2025-02-18,52500.00
1,B,AIMIRT,-4,RND05pct,SET,5000,10.20,10.60,0.00,0.00,1,2024-11-26,51000.00
2,B,CPNREIT,-9,RD40Pct,SET,5000,10.80,11.70,-0.20,-1.68,1,2025-02-27,54000.00
3,B,DIF,-23,RD50Pct,SET,5000,6.70,7.85,-0.15,-1.88,1,2022-02-13,33500.00
4,B,GVREIT,-7,RND10Pct,SET,5000,6.20,6.55,-0.05,-0.76,1,2022-02-26,31000.00
5,B,JMART,-10,RND75pct,SET50,3200,8.95,9.45,-0.20,-2.07,1,2022-02-25,28640.00
6,B,KCE,-43,RD80Pct,SET100,1000,14.00,18.30,-1.40,-7.11,1,2022-08-26,14000.00
9,B,RCL,-31,RD45pct,SET100,3000,21.25,24.40,-0.60,-2.40,1,2022-03-12,63750.00
11,B,STA,-8,RD50Pct,SET100,2500,15.00,15.80,0.00,0.00,1,2025-04-18,37500.00
12,B,TFFIF,-6,RD20Pct,SET,5000,6.00,6.30,0.05,0.80,1,2025-02-27,30000.00


In [28]:
df.query('name == "AIMIRT"')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
1,B,AIMIRT,-4,RND05pct,SET,5000,10.20,10.60,0.00,0.00,1,2024-11-26,51000.00


In [29]:
df.query('active == 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
7,B,MCS,-26,RD60Pct,SET,5000,5.80,7.10,-0.05,-0.70,0,2025-04-18,29000.00
8,B,NER,-13,RD40pct,SET,6000,4.70,4.96,0.02,0.40,0,2025-04-24,28200.00
10,B,SENA,-18,RD60pct,SET,7000,1.92,2.20,0.00,0.00,0,2024-05-10,13440.00
14,S,AWC,102,COS,SET50,9000,4.96,2.92,-0.18,-5.81,0,2024-05-09,44640.00
15,S,BCH,55,COS,SET100,2000,21.70,16.20,0.50,3.18,0,2024-08-29,43400.00
16,S,JMT,231,COS,SET50,900,50.00,11.90,-0.20,-1.65,0,2025-02-25,45000.00
17,S,PTG,69,COS,SET100,3600,11.40,7.20,-0.15,-2.04,0,2024-11-26,41040.00
18,S,PTTGC,110,RD60pct,SET50,1000,30.00,15.90,-0.20,-1.24,0,2025-03-03,30000.00
19,S,SYNEX,-459,RD70pct,SET,2000,0.00,10.90,0.30,2.83,0,2024-08-27,0.00
20,S,TMT,217,RD65pct,SET,6000,10.20,2.68,-0.08,-2.90,0,2025-03-12,61200.00


In [30]:
df.query('spd <= -40 and trade == "B"')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
6,B,KCE,-43,RD80Pct,SET100,1000,14.00,18.30,-1.40,-7.11,1,2022-08-26,14000.00


In [31]:
df.query('trade == "B" & spd > 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [32]:
df.query('trade == "S" & spd < 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
19,S,SYNEX,-459,RD70pct,SET,2000,0.00,10.90,0.30,2.83,0,2024-08-27,0.00


In [33]:
colx = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'current', 'change', '%change',
        'active', 'xdate']

In [34]:
file_name = "orders-log.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [35]:
print(f"output_file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

output_file : C:\Users\PC1\OneDrive\A5\Data\orders-log.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\orders-log.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\orders-log.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\orders-log.csv


In [36]:
df[colx].to_csv(output_file, header=True, index=False)
df[colx].to_csv(god_file, header=True, index=False)
df[colx].to_csv(icd_file, header=True, index=False)
df[colx].to_csv(osd_file, header=True, index=False)

### Convert column name to make it valid for df.query

In [38]:
df['percent'] = df['%change']

In [39]:
df.query('percent < -2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
5,B,JMART,-10,RND75pct,SET50,3200,8.95,9.45,-0.20,-2.07,1,2022-02-25,28640.00,-2.07
6,B,KCE,-43,RD80Pct,SET100,1000,14.00,18.30,-1.40,-7.11,1,2022-08-26,14000.00,-7.11
9,B,RCL,-31,RD45pct,SET100,3000,21.25,24.40,-0.60,-2.40,1,2022-03-12,63750.00,-2.40
14,S,AWC,102,COS,SET50,9000,4.96,2.92,-0.18,-5.81,0,2024-05-09,44640.00,-5.81
17,S,PTG,69,COS,SET100,3600,11.40,7.20,-0.15,-2.04,0,2024-11-26,41040.00,-2.04
20,S,TMT,217,RD65pct,SET,6000,10.20,2.68,-0.08,-2.90,0,2025-03-12,61200.00,-2.90


In [40]:
df.query('percent >= 2.00')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
15,S,BCH,55,COS,SET100,2000,21.70,16.20,0.50,3.18,0,2024-08-29,43400.00,3.18
19,S,SYNEX,-459,RD70pct,SET,2000,0.00,10.90,0.30,2.83,0,2024-08-27,0.00,2.83


In [41]:
df.loc[df.trade == 'B'].sort_values('spd',ascending=False)

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
0,B,3BBIF,-2,RD50Pct,SET,10000,5.25,5.35,0.05,0.94,1,2025-02-18,52500.00,0.94
1,B,AIMIRT,-4,RND05pct,SET,5000,10.20,10.60,0.00,0.00,1,2024-11-26,51000.00,0.00
13,B,WHAIR,-5,RD30pct,SET,5000,5.40,5.65,-0.05,-0.88,1,2022-03-03,27000.00,-0.88
12,B,TFFIF,-6,RD20Pct,SET,5000,6.00,6.30,0.05,0.80,1,2025-02-27,30000.00,0.80
4,B,GVREIT,-7,RND10Pct,SET,5000,6.20,6.55,-0.05,-0.76,1,2022-02-26,31000.00,-0.76
11,B,STA,-8,RD50Pct,SET100,2500,15.00,15.80,0.00,0.00,1,2025-04-18,37500.00,0.00
2,B,CPNREIT,-9,RD40Pct,SET,5000,10.80,11.70,-0.20,-1.68,1,2025-02-27,54000.00,-1.68
5,B,JMART,-10,RND75pct,SET50,3200,8.95,9.45,-0.20,-2.07,1,2022-02-25,28640.00,-2.07
8,B,NER,-13,RD40pct,SET,6000,4.70,4.96,0.02,0.40,0,2025-04-24,28200.00,0.40
10,B,SENA,-18,RD60pct,SET,7000,1.92,2.20,0.00,0.00,0,2024-05-10,13440.00,0.00


### Sell recently purchased stocks if profitable

In [43]:
#pd.set_option('display.max_rows', 40)
sql = """SELECT name, qty, price AS cost, date 
FROM buys B 
JOIN stocks S 
ON stock_id = S.id 
WHERE status = 'Active'
ORDER BY name, date"""
buys = pd.read_sql(sql, conpf)
buys.shape

(112, 4)

In [44]:
colu = 'name date_x qty cost price difa %change %profit'.split()
buys_price = pd.merge(buys, prices, on='name', how='inner')
buys_price['diff'] = buys_price['price'] - buys_price['cost']
buys_price['difa'] = buys_price['diff'] * buys_price['qty']
buys_price['%profit'] = round(buys_price['difa'] / (buys_price['qty'] * buys_price['cost']) * 100,2)

In [45]:
laggards = buys_price.loc[buys_price.cost > buys_price.price,colu]
laggards.shape

(109, 8)

In [46]:
watch_list = buys_price.loc[buys_price.cost <= buys_price.price,colu]
watch_list.sort_values('name',ascending=False)

,name,date_x,qty,cost,price,difa,%change,%profit
64,ORI,2025-02-17,10000,2.90,2.96,600.00,-2.63,2.07
31,GVREIT,2023-09-22,6000,6.10,6.55,2700.00,-0.76,7.38
8,AIMIRT,2024-06-17,2500,10.50,10.60,250.00,0.00,0.95


### Profit per Month

In [48]:
year = 2025
month = 2
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND MONTH(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % (year, month)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2025 AND MONTH(sells.date) = 2
ORDER BY sells.date DESC, name


In [49]:
sells_df

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days
0,TMT,2025,2,2.94,10.00,-7.06,12000,35280.00,120000.00,-84720.00,-70.60,1239
1,TMT,2025,2,2.94,10.90,-7.96,15000,44100.00,163500.00,-119400.00,-73.03,1288
2,TMT,2025,2,2.94,9.30,-6.36,9000,26460.00,83700.00,-57240.00,-68.39,993
3,ASK,2025,2,7.35,12.00,-4.65,1500,11025.00,18000.00,-6975.00,-38.75,115
4,ASK,2025,2,7.35,31.00,-23.65,4500,33075.00,139500.00,-106425.00,-76.29,732


In [50]:
print(f"Monthly Profit Amount: {sells_df.gross.sum():,.2f}")

Monthly Profit Amount: -374,760.00


In [51]:
file_name = "monthly-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [52]:
print(f"output_file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

output_file : C:\Users\PC1\OneDrive\A5\Data\monthly-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\monthly-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\monthly-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\monthly-sales.csv


In [53]:
sells_df.to_csv(output_file, header=True, index=False)
sells_df.to_csv(god_file, header=True, index=False)
sells_df.to_csv(icd_file, header=True, index=False)
sells_df.to_csv(osd_file, header=True, index=False)

In [54]:
sells_df['profit_per_day'] = (sells_df['gross'] / sells_df['buy_amt']) / sells_df['days'] * 100
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days,profit_per_day
0,TMT,2025,2,2.94,10.00,-7.06,"12,000","35,280.00","120,000.00","-84,720.00",-70.60,1239,-0.056981
1,TMT,2025,2,2.94,10.90,-7.96,"15,000","44,100.00","163,500.00","-119,400.00",-73.03,1288,-0.056698
2,TMT,2025,2,2.94,9.30,-6.36,"9,000","26,460.00","83,700.00","-57,240.00",-68.39,993,-0.068869
3,ASK,2025,2,7.35,12.00,-4.65,"1,500","11,025.00","18,000.00","-6,975.00",-38.75,115,-0.336957
4,ASK,2025,2,7.35,31.00,-23.65,"4,500","33,075.00","139,500.00","-106,425.00",-76.29,732,-0.104222


In [55]:
sells_df.gross.sum()

-374760.0

In [56]:
average_profit_percent = sells_df['profit_per_day'].mean()
print(f"Average Profit Percent: {average_profit_percent:.2f}%")

Average Profit Percent: -0.12%


In [57]:
#dt = date(2023, 11, 8)
dt = today
dt.strftime("%B %d, %Y")

'February 28, 2025'

In [58]:
print("Hello, {}! Today is {:%B %d, %Y}.".format("Alice", dt))

Hello, Alice! Today is February 28, 2025.


In [59]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-02-28 20:47:08
